# Recomendador de películas basado en contenido

## Laura Cristina López Bedoya

En esta práctica vamos a aplicar algunos de los conceptos que hemos desarrollado a lo largo de las clases de este módulo. Vamos a construir un recomendador de películas basado en contenido pero que se apoye en herramientas de procesamiento de lenguaje natural para modelar ese contenido. Utilizaremos un conjunto de datos descargado de IMDB, la base de datos de películas de internet, lo preprocesaremos para extraer la información que más nos interese y llevaremos a cabo recomendaciones basadas en la misma. Posteriormente, como gran cantidad de la información de la película la tenemos en un campo de texto libre probaremos distintas aproximaciones basadas en procesamiento de lenguaje natural.

Toda la tarea se desarrollará en este Jupyter Notebook siguiendo las instrucciones y ayudas para resolver la práctica. En cualquier caso, es necesario responder a todas y cada una de las siguientes tareas.

## Parte obligatoria

En primer lugar vamos a importar los paquetes que usaremos a lo largo de la práctica. Para representar los datos usaremos *pandas*, para realizar los calculos y medidas *numpy* y *scikit-learn*. Para el procesamiento de lenguaje natural *spacy*. EL modulo *pprint*, pretty print, sirve para imprimir de manera bonita por pantalla Se proporciona un documento con las instrucciones de instalación, por favor consultarlo antes de empezar la práctica. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from pprint import pprint

### TAREA 1. Recomendador basado en popularidad.
En primer lugar es necesario cargar y preprocesar la base de datos descargada. Por favor coloca el fichero CSV con la descarga de IMDB en el mismo directorio que este notebook y utilizar el comando *pd.read_csv* para cargarlo. Posteriormente puedes usar df.info() para visualizar una descripción de las columnas de la base de datos. Para usar pandas puedes consultar la documentación en [Pandas Docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)

In [2]:
df = pd.read_csv('IMDBdata_MainData.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273 entries, 0 to 5272
Data columns (total 27 columns):
Title             5273 non-null object
Year              5273 non-null int64
Rated             4815 non-null object
Released          5235 non-null object
Runtime           5257 non-null object
Genre             5271 non-null object
Director          5272 non-null object
Writer            5221 non-null object
Actors            5266 non-null object
Plot              5262 non-null object
Language          5263 non-null object
Country           5271 non-null object
Awards            4462 non-null object
Poster            5260 non-null object
Ratings.Source    5273 non-null object
Ratings.Value     5273 non-null object
Metascore         3876 non-null float64
imdbRating        5273 non-null float64
imdbVotes         5273 non-null object
imdbID            5273 non-null object
Type              5273 non-null object
DVD               4882 non-null object
BoxOffice         2634 non-null ob

Nos quedaremos con un subconjunto de la base de datos para el resto, en concreto, las columnas "Title","Genre","Director","Actors","Plot","Year","Rated","Runtime","imdbRating","imdbVotes" y las filas correspondientes a las 1000 películas más votadas en IMDB. Para ver las primeras filas de la tabla de datos podemos usar *df.head*. 

In [3]:
df = df[["Title","Genre","Director","Actors","Plot","Year","Rated","Runtime","imdbRating","imdbVotes"]]
df["imdbVotes"] = df["imdbVotes"].str.replace(",","").astype(float)
df = df.sort_values("imdbVotes",ascending=False)

In [4]:
df.head(10)

,Title,Genre,Director,Actors,Plot,Year,Rated,Runtime,imdbRating,imdbVotes
100,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,1994,R,142 min,9.3,1825626.0
103,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,2008,PG-13,152 min,9.0,1802351.0
112,Inception,"Action, Adventure, Sci-Fi",Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...","A thief, who steals corporate secrets through ...",2010,PG-13,148 min,8.8,1592306.0
108,Fight Club,Drama,David Fincher,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier","An insomniac office worker, looking for a way ...",1999,R,139 min,8.8,1458676.0
107,Pulp Fiction,"Crime, Drama",Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","The lives of two mob hit men, a boxer, a gangs...",1994,R,154 min,8.9,1427451.0
110,Forrest Gump,"Comedy, Drama, Romance",Robert Zemeckis,"Tom Hanks, Rebecca Williams, Sally Field, Mich...","While not intelligent, Forrest Gump has accide...",1994,PG-13,142 min,8.8,1365937.0
109,The Lord of the Rings: The Fellowship of the Ring,"Adventure, Drama, Fantasy",Peter Jackson,"Alan Howard, Noel Appleby, Sean Astin, Sala Baker",A meek Hobbit from the Shire and eight compani...,2001,PG-13,178 min,8.8,1326876.0
116,The Matrix,"Action, Sci-Fi","Lana Wachowski, Lilly Wachowski","Keanu Reeves, Laurence Fishburne, Carrie-Anne ...",A computer hacker learns from mysterious rebel...,1999,R,136 min,8.7,1314628.0
106,The Lord of the Rings: The Return of the King,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston",Gandalf and Aragorn lead the World of Men agai...,2003,PG-13,201 min,8.9,1304569.0
101,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,1972,R,175 min,9.2,1243444.0


Ahora vamos a cargar el perfil de votos de dos usuarios, simplemente ejecuta el siguiente codigo y convertiremos la lista de nombres de películas en tablas con toda la información de la base de datos.

In [5]:
user_votes1 = [ "Sin City","Match Point","Pulp Fiction",
                 "Jurassic Park","The Shawshank Redemption",
                 "Blade Runner","Annie Hall","The Godfather",
                 "The Great Escape","Psycho","Casablanca","The Social Network"]
user_votes2 = [ "Slumdog Millionaire","Grease", "Black Swan", "The Illusionist", "Ice Age",
               "Shrek", "Pretty Woman","Silver Linings Playbook"]
def get_user_profile(user_votes,df_):
    indexes = np.array([df_.loc[df_['Title'] == film_name].index.values[0] for film_name in user_votes])
    return df_.loc[indexes]

user_profile1 = get_user_profile(user_votes1,df)
user_profile2 = get_user_profile(user_votes2,df)
display("User Profile 1")
display(user_profile1)
display("User Profile 2")
display(user_profile2)

'User Profile 1'

,Title,Genre,Director,Actors,Plot,Year,Rated,Runtime,imdbRating,imdbVotes
315,Sin City,"Crime, Thriller","Frank Miller, Robert Rodriguez, Quentin Tarantino","Jessica Alba, Devon Aoki, Alexis Bledel, Power...",A film that explores the dark and miserable to...,2005,R,124 min,8.0,685825.0
3115,Match Point,"Drama, Romance, Sport",Woody Allen,"Jonathan Rhys Meyers, Alexander Armstrong, Pau...","At a turning point in his life, a former tenni...",2005,R,124 min,7.7,176391.0
107,Pulp Fiction,"Crime, Drama",Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","The lives of two mob hit men, a boxer, a gangs...",1994,R,154 min,8.9,1427451.0
263,Jurassic Park,"Adventure, Sci-Fi, Thriller",Steven Spielberg,"Sam Neill, Laura Dern, Jeff Goldblum, Richard ...","During a preview tour, a theme park suffers a ...",1993,PG-13,127 min,8.1,670652.0
100,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,1994,R,142 min,9.3,1825626.0
208,Blade Runner,"Sci-Fi, Thriller",Ridley Scott,"Harrison Ford, Rutger Hauer, Sean Young, Edwar...",A blade runner must pursue and try to terminat...,1982,R,117 min,8.2,499344.0
275,Annie Hall,"Comedy, Romance",Woody Allen,"Woody Allen, Diane Keaton, Tony Roberts, Carol...",Neurotic New York comedian Alvy Singer falls i...,1977,PG,93 min,8.1,208523.0
101,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,1972,R,175 min,9.2,1243444.0
195,The Great Escape,"Adventure, Drama, History",John Sturges,"Steve McQueen, James Garner, Richard Attenboro...",Allied prisoners of war plan for several hundr...,1963,APPROVED,172 min,8.2,177901.0
130,Psycho,"Horror, Mystery, Thriller",Alfred Hitchcock,"Anthony Perkins, Vera Miles, John Gavin, Janet...","A Phoenix secretary embezzles $40,000 from her...",1960,APPROVED,109 min,8.5,458316.0


'User Profile 2'

,Title,Genre,Director,Actors,Plot,Year,Rated,Runtime,imdbRating,imdbVotes
349,Slumdog Millionaire,Drama,"Danny Boyle, Loveleen Tandan","Dev Patel, Saurabh Shukla, Anil Kapoor, Raj Zu...",A Mumbai teen reflects on his upbringing in th...,2008,R,120 min,8.0,679975.0
3954,Grease,"Musical, Romance",Randal Kleiser,"John Travolta, Olivia Newton-John, Stockard Ch...",Good girl Sandy and greaser Danny fell in love...,1978,PG-13,110 min,7.2,181489.0
3268,Black Swan,"Drama, Thriller",Darren Aronofsky,"Natalie Portman, Mila Kunis, Vincent Cassel, B...",A committed dancer wins the lead role in a pro...,2010,R,108 min,8.0,583669.0
2991,The Illusionist,"Drama, Mystery, Romance",Neil Burger,"Edward Norton, Paul Giamatti, Jessica Biel, Ru...","In turn-of-the-century Vienna, a magician uses...",2006,PG-13,110 min,7.6,311847.0
1258,Ice Age,"Animation, Adventure, Comedy","Chris Wedge, Carlos Saldanha","Ray Romano, John Leguizamo, Denis Leary, Goran...","Set during the Ice Age, a sabertooth tiger, a ...",2002,PG,81 min,7.6,349351.0
1512,Shrek,"Animation, Adventure, Comedy","Andrew Adamson, Vicky Jenson","Mike Myers, Eddie Murphy, Cameron Diaz, John L...",After his swamp is filled with magical creatur...,2001,PG,90 min,7.9,498387.0
3207,Pretty Woman,"Comedy, Romance",Garry Marshall,"Richard Gere, Julia Roberts, Ralph Bellamy, Ja...",A man in a legal but hurtful business needs an...,1990,R,119 min,6.9,229383.0
2638,Silver Linings Playbook,"Comedy, Drama, Romance",David O. Russell,"Bradley Cooper, Jennifer Lawrence, Robert De N...","After a stint in a mental institution, former ...",2012,R,122 min,7.8,567777.0


Una vez organizada la base de datos, vamos a generar un recomendador basado en popularidad que recomiende en función del *imdbRating*. Recuerda que para ordenar los valores de un pandas dataframe se puede usar *df.sort_values* y con *df.head* mostramos los primeros valores. ¿Qué 20 películas recomendaría al usuario 1 y al usuario 2? ¿Qué podemos comentar de esas dos listas?

In [6]:
 # <<RELLENAR: Recomendador basado en popularidad>>
display("Top 20")
display(df.sort_values(by='imdbRating',ascending=False).head(20))


'Top 20'

,Title,Genre,Director,Actors,Plot,Year,Rated,Runtime,imdbRating,imdbVotes
100,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,1994,R,142 min,9.3,1825626.0
101,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,1972,R,175 min,9.2,1243444.0
102,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,1974,R,202 min,9.0,856870.0
103,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,2008,PG-13,152 min,9.0,1802351.0
351,Drishyam,"Crime, Drama, Thriller",Jeethu Joseph,"Mohanlal, Meena, Ansiba, Esther",A man goes to extreme lengths to save his fami...,2013,NOT RATED,160 min,8.9,15405.0
600,"The Good, the Bad and the Ugly",Western,Sergio Leone,"Eli Wallach, Clint Eastwood, Lee Van Cleef, Al...",A bounty hunting scam joins two men in an unea...,1966,APPROVED,178 min,8.9,539526.0
107,Pulp Fiction,"Crime, Drama",Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","The lives of two mob hit men, a boxer, a gangs...",1994,R,154 min,8.9,1427451.0
104,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,1957,APPROVED,96 min,8.9,494215.0
106,The Lord of the Rings: The Return of the King,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston",Gandalf and Aragorn lead the World of Men agai...,2003,PG-13,201 min,8.9,1304569.0
353,Anbe Sivam,"Adventure, Comedy, Drama",Sundar C.,"Kamal Haasan, Madhavan, Kiran Rathod, Nassar","Two men, one young and arrogant, the other dam...",2003,NOT RATED,160 min,8.9,8928.0


El recomendador de popularidad no entrega una lista personalizada porque no será diferente para el usuario 1 y el usuario 2. Estas recomendaciones se basan en un promedio de puntuación que le han dado los usuarios y muestra un Top, en este caso un Top 20, de las peliculas más vistas/con mayor promedio de clasificación hasta el momento. 

### TAREA 2. Recomendador basado en genero y año de la película.

En esta segunda parte vamos a extraer el género de la película y el año y vamos a construir un recomendador basado en contenido. El código es una variable categórica, para poder codificarla numericamente utilizaremos lo que se denominan dummy variables, es decir una variable binaria por cada genero. Podeis verlo explicado [aquí](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)). Además vamos a rellenar con 0's aquellos valores vacíos de la base de datos.

In [7]:
f = 'Genre'
df[f] = df[f].str.replace(' ', '')
X_dummy = df[f].str.get_dummies(',').add_prefix(f + '.')
df = df.drop([f], axis = 1)
df= pd.concat((df, X_dummy), axis = 1)
df = df.fillna(0)

df.head()

,Title,Director,Actors,Plot,Year,Rated,Runtime,imdbRating,imdbVotes,Genre.Action,...,Genre.Musical,Genre.Mystery,Genre.News,Genre.Romance,Genre.Sci-Fi,Genre.Short,Genre.Sport,Genre.Thriller,Genre.War,Genre.Western
100,The Shawshank Redemption,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,1994,R,142 min,9.3,1825626.0,0,...,0,0,0,0,0,0,0,0,0,0
103,The Dark Knight,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,2008,PG-13,152 min,9.0,1802351.0,1,...,0,0,0,0,0,0,0,0,0,0
112,Inception,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...","A thief, who steals corporate secrets through ...",2010,PG-13,148 min,8.8,1592306.0,1,...,0,0,0,0,1,0,0,0,0,0
108,Fight Club,David Fincher,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier","An insomniac office worker, looking for a way ...",1999,R,139 min,8.8,1458676.0,0,...,0,0,0,0,0,0,0,0,0,0
107,Pulp Fiction,Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","The lives of two mob hit men, a boxer, a gangs...",1994,R,154 min,8.9,1427451.0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Extract interesting columns
genre_year_matrix = df[['Year','Genre.Action', 'Genre.Adventure',
       'Genre.Animation', 'Genre.Biography', 'Genre.Comedy', 'Genre.Crime', 'Genre.Drama', 'Genre.Family', 'Genre.Fantasy',
       'Genre.Film-Noir', 'Genre.History', 'Genre.Horror', 'Genre.Music',
       'Genre.Musical', 'Genre.Mystery',  'Genre.Romance',
       'Genre.Sci-Fi',  'Genre.Sport', 'Genre.Thriller',
       'Genre.War', 'Genre.Western']].to_numpy()

Posteriormente, para construir el recomendador primero hay que construir la matriz de similitud de todos los ítems y devolver los K ítems más parecidos al perfil de cada usuario. La similitud que usaremos será la similitud del coseno usando un solo argumento, podeis ver la documentación de scikit-learn [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html). Posterioremente hay que devolver los Top-K items con mayor similaridad a los del perfil del usuario.

In [9]:
def get_content_based_recommendation(movie_df, data_matrix, user_profile,topk = 20):
    sims = cosine_similarity(data_matrix,Y=None,dense_output=True) # <<RELLENAR: Similiritud del coseno a partir de data_matrix>>
    
    # Extraer las similiritudes de los elementos puntuados
    positions = [ movie_df.index.get_loc(elem) for elem in user_profile.index.tolist()]
    sims_user = sims[positions,:]
    
    # Devuelve los topk más parecidos a los puntuados.
    shape_sims_user = sims_user.shape
    linear_recommend = np.argsort(sims_user,axis=None)[::-1]
    recommend = pd.unique(np.unravel_index(linear_recommend, shape_sims_user)[1])
    recommend = recommend[~np.in1d(recommend,positions)][:topk]
    return recommend

¿Qué 10 películas recomendaría al usuario 1 y al usuario 2? ¿Qué podemos comentar de esas dos listas?

In [10]:
recommendations1 = get_content_based_recommendation(df,genre_year_matrix,user_profile1,topk=10) # <<RELLENAR: Recomendador basado en genero y año, usuario 1>>
display('User 1')
display(df.iloc[recommendations1][["Title","imdbRating"]])
recommendations2 = get_content_based_recommendation(df,genre_year_matrix,user_profile2,topk=10) # <<RELLENAR: Recomendador basado en genero y año, usuario 2>>
display('User 2')
display(df.iloc[recommendations2][["Title","imdbRating"]])

'User 1'

,Title,imdbRating
329,The King's Speech,8.0
2088,Goal! The Dream Begins,6.8
3563,My Week with Marilyn,7.0
3219,The Iron Lady,6.4
2583,Coco Before Chanel,6.7
1557,Firewall,5.8
894,Ocean's Twelve,6.5
3456,Dead Man Walking,7.6
210,Casino,8.2
5261,Bang,6.4


'User 2'

,Title,imdbRating
3632,Free Style,4.3
2629,Stone,5.4
3221,Jonah: A VeggieTales Movie,6.6
224,Gran Torino,8.2
2522,Me and Orson Welles,6.8
4367,The Class,7.5
4911,Harrison Montgomery,7.3
4865,Childless,6.0
3459,The Secret Life of Bees,7.3
5073,Wendy and Lucy,7.1


Este recomendador basado en el genero y año de la película presenta a los usuarios una recomendación personalizada. Realiza un calculo de similitud entre las peliculas que ha visto el usuario, es decir, su perfil y las demás peliculas de la lista. 
Al tener una similitud teniendo en cuenta las variables genero y año puede dar un Top 10 de peliculas que pueden gustarle al usuario basado en sus vistas anteriores. 

### TAREA 3. Recomendador basado en el argumento. 
En esta parte construiremos un recomendador basado en el argumento. Lo haremos de dos formas, en primer lugar usando una representación del mismo basada en bolsas de palabras y posteriormente usando vectores word2vec. En primer lugar extraeremos los plots del data frame, usando el nombre de la columna y el método *to_numpy*. Alguno de los plots eran un NaN por lo que ahora es 0, lo convertimos todo en strings y en minuscula.

In [11]:
plots =df["Plot"].to_numpy()
plots_clean = [ str(plot).lower() for plot in plots]
plots_clean

['two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'when the menace known as the joker emerges from his mysterious past, he wreaks havoc and chaos on the people of gotham, the dark knight must accept one of the greatest psychological and physical tests of his ability to fight injustice.',
 'a thief, who steals corporate secrets through use of dream-sharing technology, is given the inverse task of planting an idea into the mind of a ceo.',
 'an insomniac office worker, looking for a way to change his life, crosses paths with a devil-may-care soap maker, forming an underground fight club that evolves into something much, much more.',
 "the lives of two mob hit men, a boxer, a gangster's wife, and a pair of diner bandits intertwine in four tales of violence and redemption.",
 'while not intelligent, forrest gump has accidentally been present at many historic moments, but his true love, jenny curran, eludes him.',
 'a

Para construir el bag of words vamos a utilizar la clase de *scikit-learn* [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) y la aplicamos a los plots que tenemos usando *fit_transform*. Podemos ver que palabras ha usado en la bolsa de palabras usando *get_feature_names*.

In [12]:
vectorizer = CountVectorizer(min_df=0.001,
                     max_df=0.5,
                     max_features=1000,stop_words=STOP_WORDS)

bowmovie_matrix = vectorizer.fit_transform(plots_clean) # <<RELLENAR: construir usando fit_transform>>
pprint(vectorizer.get_feature_names())

C:\Users\usuario\Anaconda3\envs\practica16\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


['000',
 '10',
 '12',
 '15',
 '1950s',
 '1960s',
 '1970s',
 'abandoned',
 'ability',
 'accident',
 'accidentally',
 'account',
 'accused',
 'act',
 'action',
 'actor',
 'actress',
 'actually',
 'adaptation',
 'adult',
 'adventure',
 'adventures',
 'affair',
 'african',
 'age',
 'aged',
 'agent',
 'agents',
 'aging',
 'ago',
 'agrees',
 'air',
 'alien',
 'aliens',
 'alive',
 'america',
 'american',
 'ancient',
 'angeles',
 'animal',
 'animals',
 'apart',
 'apartment',
 'area',
 'armed',
 'arms',
 'army',
 'arrival',
 'arrives',
 'art',
 'artist',
 'arts',
 'aspiring',
 'assassin',
 'assassination',
 'assigned',
 'assistant',
 'attack',
 'attempt',
 'attempts',
 'author',
 'avenge',
 'away',
 'awry',
 'baby',
 'bad',
 'band',
 'bank',
 'bar',
 'baseball',
 'based',
 'basketball',
 'battle',
 'beach',
 'bear',
 'beautiful',
 'befriends',
 'begin',
 'begins',
 'believe',
 'believes',
 'beloved',
 'best',
 'better',
 'big',
 'biggest',
 'birthday',
 'bizarre',
 'black',
 'blood',
 'body',
 

Si usamos esta nueva matrix de datos ¿Qué 10 películas recomendaría al usuario 1 y al usuario 2? 

In [13]:
recommendations1 = get_content_based_recommendation(df,bowmovie_matrix,user_profile1,topk=10) # <<RELLENAR: Recomendador basado en bolsa de palabras, usuario 1>>
display('User 1')
display(df.iloc[recommendations1][["Title","imdbRating"]])
recommendations2 = get_content_based_recommendation(df,bowmovie_matrix,user_profile2,topk=10)# <<RELLENAR: Recomendador basado en bolsa de palabras, usuario 2>>
display('User 2')
display(df.iloc[recommendations2][["Title","imdbRating"]])

'User 1'

,Title,imdbRating
308,Stalag 17,8.0
105,Schindler's List,8.9
4812,Sands of Iwo Jima,7.2
4844,Taxi to the Dark Side,7.6
3209,Crocodile Dundee II,5.5
4882,My Last Day Without You,5.7
2062,Lion of the Desert,8.4
3381,Tales from the Crypt: Demon Knight,6.7
2929,The Railway Man,7.1
4805,Hellraiser,7.0


'User 2'

,Title,imdbRating
493,Hum Dil De Chuke Sanam,7.6
3506,She's All That,5.8
4647,Vaalu,5.0
1262,You've Got Mail,6.6
2649,The Rage: Carrie 2,4.6
3900,Les visiteurs,7.0
3921,Dazed and Confused,7.7
576,Rab Ne Bana Di Jodi,7.2
4119,Saved!,6.9
3757,Easy A,7.1


Para esta siguiente parte vamos a cargar el modelo mediano de spacy para obtener los vectores word2vec de cada palabra de los argumentos. Si no lo teneis descargado se puede hacer en una consola como que tenga python como *python -m spacy download en_core_web_lg*, es casi 1 GB el modelo por lo que tardará un rato. Posteriormente cargamos el modelo spacy, puede tardar un poco también.

In [14]:
nlp_analyzer = spacy.load("en_core_web_lg")

C:\Users\usuario\Anaconda3\envs\practica16\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_lg' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Ahora lo usaremos para analizar la lista de descripciones que tenemos, para ello usaremos la función pipe que aplica el modelo de spacy a una lista de textos y construiremos un array con el vector de cada argumento.

In [15]:
plot_documents = list(nlp_analyzer.pipe(plots_clean))

# Construir una matrix con el promedio de vectores de cada palabra si no es una stopword
vector_matrix = np.zeros((len(plots_clean),300))
for idx, doc_ in enumerate(plot_documents):
    vector_matrix[idx,:]  =  sum([tok.vector for tok in doc_ if not tok.is_stop])/len(doc_)

Si usamos esta nueva matrix de datos ¿Qué 10 películas recomendaría al usuario 1 y al usuario 2? ¿Qué creéis que marca la diferencia entre la representación basada en bolsas de palabras y esta?

In [16]:
recommendations1 = get_content_based_recommendation(df,vector_matrix,user_profile1,topk=10)# <<RELLENAR: Recomendador basado en word2vec, usuario 1>>
display('User 1')
display(df.iloc[recommendations1][["Title","imdbRating"]])
recommendations2 = get_content_based_recommendation(df,vector_matrix,user_profile2,topk=10)# <<RELLENAR: Recomendador basado en word2vec, usuario 2>>
display('User 2')
display(df.iloc[recommendations2][["Title","imdbRating"]])

'User 1'

,Title,imdbRating
4928,Saints and Soldiers,6.8
1669,The Bounty Hunter,5.5
2929,The Railway Man,7.1
643,The 400 Blows,8.1
1487,Elizabethtown,6.4
178,Inglourious Basterds,8.3
2250,An Unfinished Life,7.0
1078,Just Go with It,6.4
1788,"I Love You, Man",7.0
570,Dhobi Ghat,7.1


'User 2'

,Title,imdbRating
1533,"Crazy, Stupid, Love.",7.4
3892,We Need to Talk About Kevin,7.5
3164,Post Grad,5.3
1679,Kicking & Screaming,5.6
2241,Wicker Park,7.0
3106,The Object of My Affection,6.0
4566,Thirteen,6.8
618,The Hunt,8.3
541,Namastey London,7.3
489,Socha Na Tha,7.6


En estos casos se recomiendan peliculas diferentes.

Para un recomendador basado en el argumento es importante realizar un análisis semántico de las peliculas, dado que su descripción puede ser fundamental para definir el gusto de una persona y no solo el genero de la pelicula. Las recomendaciones realizadas con la bolsa de palabras tiene ciertos problemas como la perdida de noción de similitud, no hay significado de la palabra ni contexto o significado de la frase, por lo que esta recomendación puede ser sesgada bajo ciertos parametros. La recomendación realizada con word2vec permite un análisis un poco más amplio dado que se puede dar una interpretación semántica aunque sigue siendo limitada. 

Estas recomendaciones ya no se basan en el promedio de puntaje dado, toman una descripción de la pelicula para darle al usuario una recomedación personalizada que le permitirá encontrar con mayor facilidad contenido que sea de su agrado.

### TAREA 4. Recomendador basado en el argumento y en Deep Learning. 
En esta parte haremos lo mismo que en la sección anterior pero usando BERT para crear los vectores que representan cada párrafo. Usaremos los modelos preentrenados de BERT que mantiene HuggingFace y que están integrados con spacy. Para instalarlo usaremos un comando parecido al anterior *python -m spacy download en_trf_bertbaseuncased_lg*. Teneis más información en el [blog de explosion.ai](https://explosion.ai/blog/spacy-transformers). La forma de cargar los vectores es igual que la usada en el apartado anterior.

In [17]:
nlp_analyzer_bert = spacy.load("en_trf_bertbaseuncased_lg")

In [19]:
# <<RELLENAR: construir la matrix bert_matrix de forma similar al vector_matrix anterior>>
plot_documents = list(nlp_analyzer_bert.pipe(plots_clean))
bert_matrix = np.zeros((len(plots_clean),768))
for idx, doc_ in enumerate(plot_documents):
    bert_matrix[idx,:]  =  sum([tok.vector for tok in doc_ if not tok.is_stop])/len(doc_)

¿Qué películas recomendaría al usuario 1 y al usuario 2 con cada aproximación? ¿Qué creéis que marca la diferencia entre esta representación y las anteriores? ¿Creéis que mejora?

In [20]:
recommendations1 = get_content_based_recommendation(df,bert_matrix,user_profile1,topk=10) # <<RELLENAR: Recomendador basado en BERT, usuario 1>>
display('User 1')
display(df.iloc[recommendations1][["Title","imdbRating"]])

recommendations2 = get_content_based_recommendation(df,bert_matrix,user_profile2,topk=10) # <<RELLENAR: Recomendador basado en BERT, usuario 2>>
display('User 2')
display(df.iloc[recommendations2][["Title","imdbRating"]])

'User 1'

,Title,imdbRating
3328,True Romance,8.0
677,Jurassic World,7.0
1078,Just Go with It,6.4
1908,Star Trek: Generations,6.6
2067,Black Water Transit,7.1
871,The Chronicles of Riddick,6.7
5135,A Fistful of Dollars,8.0
3748,Limbo,7.1
3241,Little Children,7.6
325,The Philadelphia Story,8.0


'User 2'

,Title,imdbRating
283,Cat on a Hot Tin Roof,8.1
1533,"Crazy, Stupid, Love.",7.4
967,Seventh Son,5.5
3876,Alien Zone,4.1
4085,The Way Way Back,7.4
3227,My Boss's Daughter,4.6
3665,American Heist,5.2
1027,It's Complicated,6.6
3042,Things We Lost in the Fire,7.2
792,How to Train Your Dragon 2,7.9


Todos los resultados son diferentes y es de esperarse debido a que se usan diferentes modelos para calcular el Top 10 de peliculas. 

BERT es un modelo más robusto, con mayor cantidad de datos y palabras almacenadas lo que permite realizar un análisis a profundidad comparado con word2vec y la bolsa de palabras, con el fin de encontrar una similitud más cercana con las peliculas que el usuario ha visto previamente. 

## Parte opcional: Recomendación basada en filtado colaborativo. 

La base de datos propuesta no permite el uso de técnicas de filtado colaborativo, para el que lo desee hay un dataset llamado [MovieLens](https://movielens.org/) con información de puntuaciones de usuarios. Estas primeras celdas son para cargar los datos y reorganizarlos. Deberás descomprimir el fichero *movielens-20m-dataset.zip* en el directorio de trabajo.

In [21]:
import os
from scipy.sparse.linalg import svds

In [22]:
data_path = 'movielens-20m-dataset'
movies_filename = 'movie.csv'
ratings_filename = 'rating.csv'

df_movies = pd.read_csv( os.path.join(data_path, movies_filename), usecols=['movieId', 'title'], dtype={'movieId': 'int32', 'title': 'str'})

df_ratings = pd.read_csv( os.path.join(data_path, ratings_filename), usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [23]:
display(df_movies.head(10))
display(df_ratings.head(10))

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
5,1,112,3.5
6,1,151,4.0
7,1,223,4.0
8,1,253,4.0
9,1,260,4.0


Aquí vamos a crear la matriz de ratings y quitarle la media que luego usaremos como sesgo de usuario en la recomendación. Además definimos la función que a partir de los *ratings* estimados devuelve la lista de recomendaciones.

In [24]:
df_ratings=df_ratings[:2000000]
df_movie_features = df_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

R = df_movie_features.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [25]:
def recommend_movies(preds_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1

    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])]).merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'movieId',
               right_on = 'movieId').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1]
                      

    return user_full, recommendations

### Entrenamiento
Calcula usando la función svds la descomposición en valores singulares de la matriz *R_demeaned*. la descomposición SVD devuelve dos matrices U y Vt y un escalar sigma, en este orden: U, sigma, V^T. Para calcular las matrices A (usuario-concepto) y B (concepto-item) hay que convertir el escalar sigma en una matriz diagonal D, usando *np.diag* y aplicando la siguiente formula $A=U\cdot D$, el $\cdot$ se calcula con *np.dot*. La matrix B es directamente Vt. Usaremos el valor k=50 para el número de conceptos.

In [26]:
U, sigma, Vt = svds(R_demeaned,k=50)# <<RELLENAR: Calculo de la descomposición>>
Sigma = np.diag(sigma) # <<RELLENAR: Convertir en matrix diagonal>>
A = np.dot(U,Sigma) # <<RELLENAR >>
B = Vt # <<RELLENAR >>

R_hat = np.dot(A, B) + user_ratings_mean.reshape(-1, 1)

preds_df = pd.DataFrame(R_hat, columns = df_movie_features.columns)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
0,-1.032377,0.816792,0.065367,-0.069286,-0.089829,0.488377,-0.417960,0.045866,-0.139050,-0.341772,...,0.000436,-0.000913,-0.005743,-0.006718,-0.004173,-0.002836,-0.009607,-0.006603,-0.001093,-0.002193
1,0.878419,0.018589,0.350089,0.069685,0.219143,0.379300,0.404332,0.008962,0.102663,-0.336462,...,0.001779,-0.000374,-0.001408,0.002510,0.002424,-0.000837,-0.004802,0.001166,-0.000476,0.000287
2,2.004504,0.853211,-0.124891,0.033102,-0.197886,0.725844,-0.070364,-0.043637,-0.004803,0.117869,...,0.003959,-0.004786,-0.001086,0.001811,0.005158,0.001003,0.019518,-0.001382,-0.001391,0.000514
3,-0.730047,0.575874,0.316375,-0.043769,0.175544,0.804843,0.024102,0.053929,0.182834,1.027777,...,0.001220,0.001608,0.002619,0.001078,0.003182,0.001051,0.001691,0.001607,0.002382,0.001786
4,1.487834,1.324308,1.370971,0.112844,1.282827,0.726827,1.453524,0.217864,0.285134,1.278287,...,-0.000932,-0.001432,-0.000049,0.002574,-0.002193,-0.001840,-0.003786,0.001492,0.001600,0.001771


### Predicción: 
Vamos a recomendar 10 películas al usuario 200 usando la función *recommend_movies* definida arriba. Experimenta con otros usuarios y comenta los resultados.

In [28]:
already_rated, predictions = recommend_movies(preds_df, 200, df_movies, df_ratings, num_recommendations=10)# <<RELLENAR: Recomendador basado en filtrado colaborativo>>
already_rated.head(10)

,userId,movieId,rating,title
0,200,6,5.0,Heat (1995)
11,200,150,5.0,Apollo 13 (1995)
49,200,382,5.0,Wolf (1994)
38,200,353,5.0,"Crow, The (1994)"
59,200,509,5.0,"Piano, The (1993)"
60,200,520,5.0,Robin Hood: Men in Tights (1993)
61,200,527,5.0,Schindler's List (1993)
63,200,543,5.0,So I Married an Axe Murderer (1993)
30,200,296,5.0,Pulp Fiction (1994)
27,200,273,5.0,Mary Shelley's Frankenstein (Frankenstein) (1994)


In [29]:
predictions

,movieId,title
39,47,Seven (a.k.a. Se7en) (1995)
257,288,Natural Born Killers (1994)
42,50,"Usual Suspects, The (1995)"
291,329,Star Trek: Generations (1994)
414,474,In the Line of Fire (1993)
163,185,"Net, The (1995)"
356,410,Addams Family Values (1993)
445,508,Philadelphia (1993)
483,551,"Nightmare Before Christmas, The (1993)"
365,420,Beverly Hills Cop III (1994)


In [34]:
already_rated, predictions = recommend_movies(preds_df, 156, df_movies, df_ratings, num_recommendations=10)# <<RELLENAR: Recomendador basado en filtrado colaborativo>>
already_rated.head(10)

,userId,movieId,rating,title
0,156,1,5.0,Toy Story (1995)
749,156,2000,5.0,Lethal Weapon (1987)
707,156,1892,5.0,"Perfect Murder, A (1998)"
729,156,1951,5.0,Oliver! (1968)
731,156,1953,5.0,"French Connection, The (1971)"
733,156,1956,5.0,Ordinary People (1980)
740,156,1965,5.0,Repo Man (1984)
741,156,1967,5.0,Labyrinth (1986)
753,156,2006,5.0,"Mask of Zorro, The (1998)"
681,156,1792,5.0,U.S. Marshals (1998)


In [35]:
predictions

,movieId,title
531,920,Gone with the Wind (1939)
2338,4011,Snatch (2000)
998,1721,Titanic (1997)
533,924,2001: A Space Odyssey (1968)
86,151,Rob Roy (1995)
1653,2872,Excalibur (1981)
2548,4361,Tootsie (1982)
1783,3100,"River Runs Through It, A (1992)"
2043,3524,Arthur (1981)
783,1347,"Nightmare on Elm Street, A (1984)"
